In [1]:
import pandas as pd

In [6]:
source_comments = pd.read_csv('./wall/girl_wall/comments/21.02.19-12:08_girl.csv', index_col=0)
source_answers = pd.read_csv('./wall/girl_wall/answers/21.02.19-12:08_girl.csv', index_col=0)
source_posts = pd.read_csv('./wall/girl_wall/posts/21.02.19-12:08_girl.csv', index_col=0)
source_comments.replace({pd.np.NaN: None}, inplace=True)
source_answers.replace({pd.np.NaN: None}, inplace=True)
source_posts.replace({pd.np.NaN: None}, inplace=True)
source_comments.sort_values(by=['post_id', 'date'] , inplace=True)
source_answers.sort_values(by=['post_id', 'date'] , inplace=True)

In [10]:
#base
def get_dialogs_from_answers():
    from json import JSONDecoder
    j = JSONDecoder()
    
    already_used_id = []
    dialogs = []

    answers = source_answers.copy()
    anchor = answers[list(map(lambda x: x!='[]', answers['parents_stack']))].head(1)
    while not anchor.empty:
        dialog = []
        already_used_id.append(int(anchor['comment_id']))

        parent_id = j.decode(answers['parents_stack'].iloc[0])[0]
        dialog.append(source_comments[source_comments['comment_id'] == parent_id].head(1))
        dialog.append(anchor)

        after_anchor = answers[answers['reply_to_comment'] == int(anchor['comment_id'])].head(1)
        while not after_anchor.empty:
            anchor = after_anchor
            already_used_id.append(anchor['comment_id'])
            dialog.append(anchor)
            after_anchor = answers[answers['reply_to_comment'] == int(anchor['comment_id'])].head(1)

        if len(pd.concat(dialog)) > 1:
            dialogs.append(pd.concat(dialog))

        answers = answers[~answers['comment_id'].isin(already_used_id)]
        anchor = answers[list(map(bool, answers['parents_stack']))].head(1)
    return dialogs
answers_dialogs = get_dialogs_from_answers()

In [12]:
def answer_post_processing(dialogs):
    res=[]
    dialog=[]
    dialog.append(dialogs[0])
    for cur in dialogs[1:]:
        if set(dialog[-1]['from_id']) == set(cur['from_id'])\
            and set(dialog[-1]['post_id']) == set(cur['post_id']):
            dialog.append(cur)
        else:
            if len(dialog) > 1:
                head = dialog[0]
                tail = pd.concat(dialog[1:])
                tail = tail[tail['parents_stack']!='[]']
                res.append(pd.concat([head, tail]))
            else:
                res.append(pd.concat(dialog))
            dialog=[cur]
    return res
proc_answers_dialogs = answer_post_processing(answers_dialogs)

In [13]:
def get_dialogs_from_comments():
    already_used_id = []
    dialogs = []
    comments = source_comments.copy()

    comments = comments[~comments['comment_id'].isin(already_used_id)]
    anchor = comments[list(map(bool, comments['reply_to_comment']))].head(1)
    while(not anchor['reply_to_comment'].dropna().empty):
        dialog = []
        comments = comments[~comments['comment_id'].isin(already_used_id)]
        anchor = comments[list(map(bool, comments['reply_to_comment']))].head(1)
        already_used_id.append(int(anchor['comment_id']))
        first = comments[comments['comment_id'] == int(anchor['reply_to_comment'])].head(1)
        dialog.append(first)
        dialog.append(anchor)
        after_anchor = comments[comments['reply_to_comment'] == int(anchor['comment_id'])].head(1)
        filter_user = (comments['from_id'] == int(anchor['reply_to_user']))\
                        | (comments['from_id'] == int(anchor['from_id']))
        filter_post = comments['post_id'] == int(anchor['post_id'])
        while(not after_anchor.dropna().empty):
            anchor = after_anchor
            already_used_id.append(int(anchor['comment_id']))
            dialog.append(anchor)
            filter_reply = comments['reply_to_comment'] == int(anchor['comment_id'])
            after_anchor = comments[filter_reply & filter_user & filter_post].head(1)

        if len(pd.concat(dialog)) > 1:
            dialogs.append(pd.concat(dialog))
        comments = comments[~comments['comment_id'].isin(already_used_id)]
        anchor = comments[list(map(bool, comments['reply_to_comment']))].head(1)
    return dialogs
comments_dialogs = get_dialogs_from_comments()

In [14]:
def comments_post_processing(dialogs):
    res=[]
    dialog=[]
    dialog.append(dialogs[0])
    for cur in dialogs[1:]:
        if set(dialog[-1]['from_id']) == set(cur['from_id'])\
            and set(dialog[-1]['post_id']) == set(cur['post_id']):
            dialog.append(cur)
        else:
            res.append(pd.concat(dialog))
            dialog=[cur]
    return res


In [15]:
concat_dialogs = proc_answers_dialogs.copy()
concat_dialogs.extend(proc_comments_dialogs)

In [21]:
def dialogs_to_json(dialogs, path_to_save):
    from json import JSONEncoder
    
    j = JSONEncoder()
    all_dialogs = []
    persona_id = 0
    other_id = 0
    
    for num, i in enumerate(dialogs):
        dialog = []
        for _, row in i.iterrows():
            if row['from_id']==row['owner_id']:
                persona_type = 'persona'
                persona_id = int(row['from_id'])
            else:
                persona_type = 'other'
                other_id = int(row['from_id'])
            utterance = {
                "persona_type": persona_type,
                "num_answers": int(row['num_answer']),
                "text": row['text']
            }
            dialog.append(utterance)
        
        post = source_posts[source_posts['post_id'] == int(i['post_id'].iloc[0])].head(1)
        curr_dialog = {
            "dialog_id": f"270219_{str(num)}",
            "post_text": post['text'].iloc[0],
            "post_num_comments": int(post['num_comments'].iloc[0]),
            "persona_id": persona_id,
            "other_id": other_id,
            "dialog": dialog
        }
        all_dialogs.append(curr_dialog)
    with open(path_to_save, 'w') as fw:
        fw.write(j.encode(all_dialogs))

In [22]:
dialogs_to_json(concat_dialogs, './result_json/girl_270219.json')

In [44]:
def view_dialog(table):
    if len(table) > 1:
        res = []
        for _, i in table.iterrows():
            if i['text']:
                res.append(i['text'])
            else:
                return
        return res
    
with open('./girl_270219.txt', 'w') as fw:
    for num, i in enumerate(concat_dialogs):
        a = view_dialog(i)
        if a:
            fw.write('*'*10+'\n\n\n\n'+str(num)+"  "+"*"*10+'\n')
            for ln, line in enumerate(a):
                fw.write(str(ln)+" "+line+'\n')
    fw.write('*'*10+'\n\n\n\n'+str(num)+"  "+"*"*10+'\n')

In [37]:
a = pd.DataFrame({'num_utt': list(map(len, concat_dialogs))})

In [43]:
a.stack().value_counts().to_dict()

{2: 4681,
 3: 2552,
 4: 797,
 5: 275,
 6: 160,
 7: 78,
 8: 59,
 9: 34,
 10: 28,
 11: 17,
 12: 11,
 14: 10,
 13: 7,
 16: 3,
 20: 3,
 18: 2,
 23: 2,
 21: 2,
 30: 2,
 26: 1,
 15: 1,
 33: 1,
 48: 1,
 40: 1,
 19: 1}